In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
import re
import dateparser 

In [ ]:
# here we scrape information on president's term and party
# 
# NOTE: Britiania seems to be attempting to block web scrapers. When this happened, a regular requests
# approach will fail. To bypass this, you will need to use  selenium. 
# The following code should work if site is blocking scraper:

# Start a driver session....

    # if you have selenium 3 installed, use one of these:
#driver = webdriver.Firefox(executable_path=GeckoDriverManager().install()) # this will work on Windows and Mac, and should work on Linux when run the first time
#driver = webdriver.Firefox(executable_path=<insert path to manual downloaded geckodriver>)
driver = webdriver.Firefox() # use if geckodriver is in your PATH environmnet variable (which includes the same folder as your notebook)

    # if you hve selenium 4 installed, use one of these:
#driver = webdriver.Firefox(service=Service(GeckoDriverManager().install())) # this will work on Windows and Mac, and should work on Linux when run the first time
#driver = webdriver.Firefox() # use if geckodriver is in your PATH environmnet variable (which includes the same folder as your notebook)

driver.get("https://www.britannica.com/topic/Presidents-of-the-United-States-1846696")
driver.implicitly_wait(10)
page_source = driver.page_source
driver.close() 

In [ ]:
# pandas read html will parse the contents of the table in the downloaded webpage
presidents = pd.read_html(page_source)[0]
presidents

In [ ]:
# note that the last two rows contains non-presidential information
# let's remove these last two rows...
presidents = presidents.drop([int(len(presidents)-1), int(len(presidents)-2)])
presidents

In the following cell, we create two new columns - from and to. We parse the term column for the information we need, and format appropriately.

In [ ]:
# first, split the string in the term column using dash as delimiter - store this in new column called 'from'
presidents['from'] = presidents['term'].apply(lambda x: dateparser.parse(x.split("–")[0]).year)

# calculate 'to' based on the content of the term string
def to_year(row):    
    row['term'] = re.sub(r"[^\d-]", "", row['term']) # replace any non-digit before dash with blank
    term_list = row['term'].split("–") # split on dash (to get start and end year)
    if  len(term_list)== 1: # if we only have one date, then this is both from and to
        return row['from']
    elif len(term_list) == 2:
        return row['from'][:2] + term_list[1] # return first two digits of from with string in to field
    else:
        return "bad data"
    return row
    
presidents['to'] = presidents.apply(lambda row: to_year(row), axis=1)

presidents

In [ ]:
presidents.to_csv("./data/presidential_party_and_term.csv", index=False)